In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
    "conf": {
        "spark.jars.packages": "io.delta:delta-core_2.12:1.0.0",
        "spark.sql.extensions":"io.delta.sql.DeltaSparkSessionExtension",
        "spark.sql.catalog.spark_catalog":"org.apache.spark.sql.delta.catalog.DeltaCatalog"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,,,None,
3,None,pyspark,idle,,,None,✔


In [3]:
df = spark.read.csv('s3://pags-avengers-dev/agg_data/spotify/dat_load=20230223/top_100_streamed_songs.csv',inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------+
|                 _c0|                 _c1|     _c2|   _c3|_c4|     _c5| _c6|        _c7|         _c8|             _c9|    _c10|   _c11|   _c12|        _c13|
+--------------------+--------------------+--------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------+
|                  id|                name|duration|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|danceability|
|4ZtFanR9U6ndgddUv...|Good 4 U Olivia R...|    2.97| 0.664|  9|  -5.044|   1|      0.154|       0.335|             0.0|  0.0849|  0.688|166.928|       0.563|
|5fxyZf6m2xHeSrOzU...|Stay The Kid LARO...|     2.3| 0.506|  8| -11.275|   1|     0.0589|       0.379|           0.868|    0.11|  0.454|170.054|       0.564|
|5nujrmhLynf4yMoMt...|Levitating Dua Li...|    3.38|

In [5]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '-- packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
unterminated string literal (detected at line 2) (<stdin>, line 2)
  File "<stdin>", line 2
    os.environ['PYSPARK_SUBMIT_ARGS'] = '-- packages com.amazonaws:aws-java-sdk:1.7.4,org.apache.hadoop:hadoop-
                                        ^
SyntaxError: unterminated string literal (detected at line 2)



In [6]:
df.write.format("delta") \
    .mode("overwrite") \
    .save("s3://pags-avengers-dev/agg_data/spotify_delta/dat_load=20230223/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from delta.tables import *
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
deltaTable = DeltaTable.forPath(spark, "s3://pags-avengers-dev/agg_data/spotify_delta/dat_load=20230223/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
deltaTable.toDF().show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'SparkSession' object has no attribute '_wrapped'
Traceback (most recent call last):
  File "/tmp/spark-97cb2436-de89-4692-96c0-8607599a8cda/userFiles-c40a6946-efbd-4ce9-8872-afb6e7d3b3d4/io.delta_delta-core_2.12-1.0.0.jar/delta/tables.py", line 47, in toDF
    return DataFrame(self._jdt.toDF(), self._spark._wrapped)
AttributeError: 'SparkSession' object has no attribute '_wrapped'



In [13]:
df1 = (
    spark.read.format("delta")
    .load("s3://pags-avengers-dev/agg_data/spotify_delta/dat_load=20230223/")
)
df1.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------+
|                 _c0|                 _c1|     _c2|   _c3|_c4|     _c5| _c6|        _c7|         _c8|             _c9|    _c10|   _c11|   _c12|        _c13|
+--------------------+--------------------+--------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+------------+
|                  id|                name|duration|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|danceability|
|4ZtFanR9U6ndgddUv...|Good 4 U Olivia R...|    2.97| 0.664|  9|  -5.044|   1|      0.154|       0.335|             0.0|  0.0849|  0.688|166.928|       0.563|
|5fxyZf6m2xHeSrOzU...|Stay The Kid LARO...|     2.3| 0.506|  8| -11.275|   1|     0.0589|       0.379|           0.868|    0.11|  0.454|170.054|       0.564|
|5nujrmhLynf4yMoMt...|Levitating Dua Li...|    3.38|